In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import powerlaw
from tqdm import tqdm
from matplotlib.colors import LinearSegmentedColormap
import colorsys
from matplotlib import animation, rc
from IPython.display import HTML
from matplotlib.animation import FuncAnimation, PillowWriter, FFMpegWriter
import random

In [ ]:
def infect_cure_node(G,state,active_nodes,alpha,beta):
    #change the state of the nodes
    new_state=state
    for a_node_id in active_nodes:
        
        if state[a_node_id]==1:
            #node can be cured
            if random.random()<alpha:
                new_state[a_node_id]=0
            else:
            #or node can infect neighbours
                neighbors=[n for n in G.neighbors(a_node_id) if state[n]==0]
                for neigh_index in neighbors:
                        if random.random()<beta:
                            new_state[neigh_index]=1
        #node can be infected by one of the neighbours
        elif state[a_node_id]==0:
            neighbors=[n for n in G.neighbors(a_node_id) if state[n]==1]
            for neigh_index in neighbors:
                    if random.random()<beta:
                        new_state[a_node_id]=1
                        break

    return new_state
def create_connections(G,active_nodes):
    #create connections for active nodes
    active_dict={}
    for a_node_id in active_nodes:
        active_dict[a_node_id]=1
        connections=[(a_node_id,random.randint(0,len(nodes)-1)) for ii in range(m)]
        G.add_edges_from(connections)
    nx.set_node_attributes(G, values=active_dict, name='activity_status')
    return G


#set number of nodes and number of links created by each node
alpha=0.03
beta=0.015
m=2
n=1000
max_iter=5000
#set prescaler which helps the network with creating connections
eta=100

#create graph with no links
nodes=np.arange(0,n)
G=nx.Graph()
G.add_nodes_from(nodes)

#assign the activ_status to each node
activity_distribution = powerlaw.Power_Law(
    xmin=0.001,xmax=1.0, parameters=[2.8], discrete=False
)
activity= eta*activity_distribution.generate_random(n)

activity_status=dict(zip(np.arange(0,n,1),([0]*n)))
nx.set_node_attributes(G, values=activity_status, name='activity_status')

#set initial infected and healthy status
q=0.3
node_status=dict(zip(np.arange(0,n,1),(np.random.choice([0,1],n,p=[1-q,q]))))
nx.set_node_attributes(G, values=node_status, name='state')

#list of different timeframes of network
graph_list=[]
graph_list+=[G]
infected_list=[]
state=node_status

for ii in tqdm(range(max_iter)):
    G=nx.Graph()
    G.add_nodes_from(nodes)
    nx.set_node_attributes(G, values=activity_status, name='activity_status')
    
    #select nodes which will be active
    active_nodes=[node_id  for node_id, aa in zip(nodes,activity) if np.random.random()<=min(1,aa)]

    create_connections(G,active_nodes)
    
    state=infect_cure_node(G,state,active_nodes,alpha,beta)   
    nx.set_node_attributes(G, values=state, name='state')
    
    infected_list+=[np.sum(list(state.values()))]
    graph_list+=[G]

In [ ]:
plt.plot(infected_list)

In [ ]:

infected_vs_lambda=[]
#set number of nodes and number of links created by each node
m=2
n=1000
max_iter=5000
avg_iter=5
#set prescaler which helps the network with creating connections
eta=100
nodes=np.arange(0,n)
activity= eta*activity_distribution.generate_random(n)
# betas=np.arange(0.0001,0.020,0.005).tolist()+np.arange(0.02,0.025,0.001).tolist()+np.arange(0.03,0.06,0.005).tolist()
betas=[0.001,0.005,0.008,0.01,0.012,0.014,0.015,0.018,0.019,0.020,0.021,0.022]#,0.025,0.03,0.035,0.04,0.045,0.05]
# betas=[0.001,0.03]
# betas=[0.03]
alpha=0.03
for beta in tqdm(betas):
    average_steady_infected=[]
    q=0.3
    node_status=dict(zip(np.arange(0,n,1),(np.random.choice([0,1],n,p=[1-q,q]))))
    for jj in np.arange(avg_iter):
       
        #create graph with no links
        nodes=np.arange(0,n)
        G=nx.Graph()
        G.add_nodes_from(nodes)



        activity_status=dict(zip(np.arange(0,n,1),([0]*n)))
        nx.set_node_attributes(G, values=activity_status, name='activity_status')

        #set initial infected and healthy status
       
        nx.set_node_attributes(G, values=node_status, name='state')

        #list of different timeframes of network
       
        infected_list=[]
        state=node_status

        for ii in range(max_iter):
            G=nx.Graph()
            G.add_nodes_from(nodes)
            nx.set_node_attributes(G, values=activity_status, name='activity_status')
            
            #select nodes which will be active
            active_nodes=[node_id  for node_id, aa in zip(nodes,activity) if np.random.random()<=min(1,aa)]

            G=create_connections(G,active_nodes)
            
            state=infect_cure_node(G,state,active_nodes,alpha,beta)   
            nx.set_node_attributes(G, values=state, name='state')
            
            infected_list+=[np.sum(list(state.values()))]
            plt.plot(infected_list)
        average_steady_infected+=[np.average(infected_list[(max_iter-200):])]
        print(average_steady_infected)
    infected_vs_lambda+=[np.average(average_steady_infected)]

In [ ]:
activity= eta*activity_distribution.generate_random(10**6)

In [ ]:
np.std(activity)

In [ ]:
np.mean(activity)

In [ ]:
1/m*1/(np.std(activity)+np.mean(activity))

In [ ]:
plt.plot(np.array(betas)/alpha,infected_vs_lambda)

In [ ]:
infected_vs_lambda

In [ ]:
def plot_graph(G,pos,cmap):
    """
    Plot the graph for SIS model.


    Arguments:
    ----------
    G -- networkx.Graph or networkx.DiGraph instance
        graph to plot

    pos -- layout selected for plotting the network

    cmap -- color map which indicates different states of nodes
    
    Returns:
    --------
    plotted network
    """
    plt.clf()
    # plt.figure(figsize=(16,9))
    #assignes different node size based on node degree
    actstatus=np.array(list(nx.get_node_attributes(G,'activity_status').values()))>0
    node_color = np.array(list(nx.get_node_attributes(G,'state').values()))
    node_size=np.array([20+v*100 for v in dict(G.degree).values()])

    nx.draw(G,pos=pos , node_color =node_color, nodelist=nodes, vmin=0, vmax=2,node_shape='o', cmap =cmap, node_size=node_size,alpha=0.4)
    nx.draw(G,pos=pos , node_color =node_color[actstatus], nodelist=nodes[actstatus], vmin=0, vmax=2,node_shape='o', cmap =cmap, node_size=node_size[actstatus])
    red_circle = mlines.Line2D(range(1),range(1),color="white", marker='o', markerfacecolor="red",markersize=10)
    green_circle = mlines.Line2D(range(1),range(1),color="white", marker='o', markerfacecolor="green",markersize=10)
    plt.legend((green_circle,red_circle),('Healthy','Sick'))
    return(plt)

   

In [ ]:

#create custom color map to disinguish between sick, healthy and vaccinated
pos=nx.spring_layout(G,k=0.03)
colors = [ (0.1, 0.9, 0),(0.9, 0.1, 0)]
cmap = LinearSegmentedColormap.from_list('cmap_own', colors, N=2)
plt.figure(figsize=(16,9))
plot_graph(graph_list[299],pos,cmap)


In [ ]:
def init_graph():
    # First set up the figure, the axis, and the plot element we want to animate
    plt.title('Time evolution of SIS model with vaccination')
    plt.plot()

# animation function. This is called sequentially
def animate_graph(ii):
    plot_graph(graph_list[ii],pos,cmap)


# call the animator
fig = plt.figure(figsize=((16,9)))
anim = animation.FuncAnimation(fig, animate_graph, init_func=init_graph, interval=100,frames=len(graph_list))
rc('animation', html='html5')
writer = FFMpegWriter(fps=5, metadata=dict(artist='Me'),  bitrate=10000)
anim.save('time_evolution_of_graph.mp4', writer=writer)

In [ ]:
H=nx.Graph()
H.add_nodes_from(nodes)
for graph in graph_list[:30]:
     # H.add_edges_from(list(H.edges())+list(graph.edges()))
     H=nx.compose(H,graph)
nx.draw(H,pos=pos,node_shape='o',node_size=[10+v*100 for v in dict(G.degree).values()])

In [ ]:
node_list=list(dict(H.degree).values())
vals,counts=np.unique(node_list,return_counts=True)
vals,counts=np.unique(counts,return_counts=True)

prob=counts/np.sum(counts)
cum_sum=np.cumsum(prob[::-1])[::-1]


plt.figure(figsize=(16,9),dpi= 100)
plt.loglog(vals,cum_sum,"^-",label="Cumulated P(k)")

logx = np.log(vals)
logy = np.log(cum_sum)
coeffs = np.polyfit(logx,logy,deg=1)
poly = np.poly1d(coeffs)
yfit = lambda x: np.exp(poly(np.log(x)))
plt.loglog(vals,prob,"o-",label="P(k)")
plt.loglog(vals,yfit(vals))
plt.legend()
plt.grid()
plt.xlabel("k",fontsize=15)
plt.ylabel("P(k)",fontsize=15)
plt.title(f"Wykres dla m={m}, $\gamma$={np.round(coeffs[0],2)}")